In [4]:
import pandas as pd
import os
import collections

In [5]:
ipsn_df = pd.read_csv('data/Clean_TPC/ipsn_clean.csv')
mobicom_df = pd.read_csv('data/Clean_TPC/mobicom_clean.csv')
mobihoc_df = pd.read_csv('data/Clean_TPC/mobihoc_clean.csv')
sensys_df = pd.read_csv('data/Clean_TPC/sensys_clean.csv')
sigcomm_df = pd.read_csv('data/Clean_TPC/sigcomm_clean.csv')

In [7]:
def create_year_dict(conf_df):
    "Given the conf df , return a sorted list of years of that conference"
    years = list(set(conf_df['Year']))
    years.sort()
    return years

def retention(conf_df):
    years = create_year_dict(conf_df)
    year_col = []
    retention_col = []
    
    for index, year in enumerate(years):
        
        if index > 1 and index < len(years):
            
            current_year = years[index]
            last_year = years[index -1]
            last_last_year = years[index -2]
            
            current_df = conf_df[conf_df['Year'] == current_year]
            current_tpc = set(current_df['University/Organization'])
            
            last_df = conf_df[conf_df['Year'] == last_year]
            last_tpc = set(last_df['University/Organization'])
            
            last_last_df = conf_df[conf_df['Year'] == last_last_year]
            last_last_tpc = set(last_last_df['University/Organization'])
            
            past_tpc = set()
            past_tpc.update(last_tpc)
            past_tpc.update(last_last_tpc)
            
            retention = current_tpc.intersection(past_tpc)
            ret_percent = len(retention) / len(current_tpc) * 100
            
            year_col.append(current_year)
            retention_col.append(ret_percent)
            
    return year_col, retention_col
    
            
def create_df(conf, conf_df):
    "Given the conf df makes the final dataframe for each conference"
    year, ret = retention(conf_df)
    conf_name = [str(conf)] * len(ret)
    
    return pd.DataFrame(list(zip(conf_name, ret, year)), columns = ['Name', 'Retention(%over2yrs)', 'Year'])

ipsn_ret = create_df('ipsn', ipsn_df)
mobihoc_ret =  create_df('mobihoc', mobihoc_df)
mobicom_ret = create_df('mobicom', mobicom_df)
sensys_ret = create_df('sensys', sensys_df)
sigcomm_ret = create_df('sigcomm', sigcomm_df)

min_y = [ipsn_ret['Year'].min(),mobihoc_ret['Year'].min(), mobicom_ret['Year'].min(), sensys_ret['Year'].min(), 
         sigcomm_ret['Year'].min()]

max_y = [ipsn_ret['Year'].max(),mobihoc_ret['Year'].max(), mobicom_ret['Year'].max(), sensys_ret['Year'].max(), 
         sigcomm_ret['Year'].max()]

#ipsn and mobicom

print(min_y)
mobihoc_ret.head(4)

[2006, 2002, 2002, 2005, 2002]


,Name,Retention(%over2yrs),Year
0,mobihoc,53.571429,2002
1,mobihoc,44.117647,2003
2,mobihoc,30.232558,2004
3,mobihoc,37.288136,2005


In [8]:
index = mobihoc_ret[mobihoc_ret['Year'] < 2006].index
mobihoc_ret.drop(index, inplace = True)
mobihoc_ret

index = mobicom_ret[mobicom_ret['Year'] < 2006].index
mobicom_ret.drop(index, inplace = True)


index = sensys_ret[sensys_ret['Year'] < 2006].index
sensys_ret.drop(index, inplace = True)


index = sigcomm_ret[sigcomm_ret['Year'] < 2006].index
sigcomm_ret.drop(index, inplace = True)


In [9]:
retention_df = pd.concat([ipsn_ret, mobihoc_ret, mobicom_ret, sensys_ret, sigcomm_ret])
retention_df.to_csv('data/retention_uni.csv', header = True, index = False)

In [14]:
retention_df = pd.read_csv('data/retention_uni.csv')
index = retention_df[retention_df['Name'] == 'ipsn'].index
# retention_df.drop(index, inplace = True)
# set(retention_df['Name'])
index
retention_df.loc[retention_df['Name'].isin(['mobicom', 'sensys'])]

,Name,Retention(%over2yrs),Year
28,mobicom,53.846154,2006
29,mobicom,55.102041,2007
30,mobicom,41.509434,2008
31,mobicom,30.158730,2009
32,mobicom,44.897959,2010
33,mobicom,51.111111,2011
34,mobicom,58.139535,2012
35,mobicom,44.000000,2013
36,mobicom,42.622951,2014
37,mobicom,45.000000,2015


In [51]:
import plotly.graph_objects as go
import plotly
import pandas as pd

# plotly.offline.init_notebook_mode(connected = True)

df = pd.DataFrame({'Machine': ['K2K01','K2K01','K2K01','K2K02','K2K02','K2K02','K2K03','K2K03','K2K03'],
                   'Units': [100,200,400,400,300,100,500,700,500],
                   'Time':[11,12,13,11,12,13,10,11,12]})

groups = df.groupby(by='Machine')

# data = []
# colors=['red', 'blue', 'green']

# for group, dataframe in groups:
#     dataframe = dataframe.sort_values(by=['Time'])
#     trace = go.Scatter(x=dataframe.Time.tolist(), 
#                        y=dataframe.Units.tolist(),
#                        marker=dict(color=colors[len(data)]),
#                        name=group)
#     data.append(trace)

# layout =  go.Layout(xaxis={'title': 'Time'},
#                     yaxis={'title': 'Produced Units'},
#                     margin={'l': 40, 'b': 40, 't': 50, 'r': 50},
#                     hovermode='closest')

# figure = go.Figure(data=data, layout=layout)  
# figure.show()
df

,Machine,Units,Time
0,K2K01,100,11
1,K2K01,200,12
2,K2K01,400,13
3,K2K02,400,11
4,K2K02,300,12
5,K2K02,100,13
6,K2K03,500,10
7,K2K03,700,11
8,K2K03,500,12


In [5]:
import plotly.express as px
data = dict(
    character=["Eve", "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
    parent=["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
    value=[10, 14, 12, 10, 2, 6, 6, 4, 4])

fig =px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
)
# fig.show()


In [15]:
char = data['character']
par = data['parent']
val = data['value']
pd.DataFrame(list(zip(char, par, val)))

,0,1,2
0,Eve,,10
1,Cain,Eve,14
2,Seth,Eve,12
3,Enos,Seth,10
4,Noam,Seth,2
5,Abel,Eve,6
6,Awan,Eve,6
7,Enoch,Awan,4
8,Azura,Eve,4


In [6]:
ipsn_df = pd.read_csv('data/Clean_Authors/ipsn_authors.csv')

mobicom_df = pd.read_csv('data/Clean_Authors/mobicom_authors.csv')

mobihoc_df = pd.read_csv('data/Clean_Authors/mobihoc_authors.csv')

sigcomm_df = pd.read_csv('data/Clean_Authors/sigcomm_authors.csv')

sensys_df = pd.read_csv('data/Clean_Authors/sensys_authors.csv')

ipsn_tpc = pd.read_csv('data/Clean_TPC/ipsn_clean.csv')

In [17]:
set1 = {2, 4, 5, 6}  
set2 = {4, 6, 7, 8}  
set3 = {4,8} 
set1 - set3

{2, 5, 6}

In [21]:
authors = set(ipsn_df['Author_Name'])
tpc = set(ipsn_tpc['Name'])

tpc_auth = list(authors.intersection(tpc))
# tpc_status = ['TPC Member'] * len(tpc_auth)

tpc_auth_df = pd.DataFrame(list(zip(['TPC-Member'], [len(tpc_auth)])), columns = ['Status', 'Count'])

other_auth = list(authors - tpc)
other_auth_df = pd.DataFrame(list(zip(['Others'], [len(other_auth)])), columns = ['Status', 'Count'])

pd.concat([tpc_auth_df, other_auth_df])



,Status,Count
0,TPC-Member,59
0,Others,1633


In [7]:
df = pd.read_csv('data/Clean_Authors/ipsn_authors.csv')
df = df[df['Year'] == 2008]
uni_count = dict(collections.Counter(df['University/Organization']))
top = ['Microsoft Research', 'Stanford']

rank = []
no_of_authors = []
for key in uni_count.keys():
    if key in top:
        rank.append('Top 20')
        no_of_authors.append(uni_count[key])
    else:
        rank.append('Other')
        no_of_authors.append(uni_count[key])
        
# pd.DataFrame(list(zip(rank, no_of_authors)), columns = ['Rank', 'No_of_authors'])

In [ ]:
import plotly.express as px
# This dataframe has 244 lines, but 4 distinct values for `day`
df = px.data.tips()
fig = px.pie(df, values='tip', names='day')
fig.show()

## Extracting the years for the conference

In [19]:
def conf_years(conf, conf_df):
    "Given the conf name and the dataframe for the same returns a list of the years for that conference "
    year_list = list(set(conf_df['Year']))
    year_list.sort()
    
    new_year_list = []
    for year in year_list:
        year = str(conf) + "_" + str(year)
        new_year_list.append(year)
    
    return new_year_list

ip = conf_years('ipsn', ipsn_df)
ip

['ipsn_2004',
 'ipsn_2006',
 'ipsn_2007',
 'ipsn_2008',
 'ipsn_2010',
 'ipsn_2011',
 'ipsn_2013',
 'ipsn_2015',
 'ipsn_2017',
 'ipsn_2018',
 'ipsn_2019']

In [21]:
year_dict = collections.defaultdict(list)

year_dict['ipsn'] = conf_years('ipsn', ipsn_df)

year_dict['mobicom'] = conf_years('mobicom', mobicom_df)

year_dict['mobihoc'] = conf_years('mobihoc', mobihoc_df)

year_dict['sensys'] = conf_years('sensys', sensys_df)

year_dict['sigcomm'] = conf_years('sigcomm', sigcomm_df)

In [22]:
year_dict

defaultdict(list,
            {'ipsn': ['ipsn_2004',
              'ipsn_2006',
              'ipsn_2007',
              'ipsn_2008',
              'ipsn_2010',
              'ipsn_2011',
              'ipsn_2013',
              'ipsn_2015',
              'ipsn_2017',
              'ipsn_2018',
              'ipsn_2019'],
             'mobicom': ['mobicom_2000',
              'mobicom_2001',
              'mobicom_2002',
              'mobicom_2003',
              'mobicom_2004',
              'mobicom_2005',
              'mobicom_2006',
              'mobicom_2007',
              'mobicom_2008',
              'mobicom_2010',
              'mobicom_2011',
              'mobicom_2012',
              'mobicom_2013',
              'mobicom_2014',
              'mobicom_2015',
              'mobicom_2016',
              'mobicom_2017',
              'mobicom_2018',
              'mobicom_2019'],
             'mobihoc': ['mobihoc_2001',
              'mobihoc_2002',
              'mobihoc_

In [16]:
for year in year_dict['ipsn']:
    year = str(year) + '_' + str('ipsn')
    year_dict[year]



2004
***
2006
***
2007
***
2008
***
2010
***
2011
***
2013
***
2015
***
2017
***
2018
***
2019
***


In [3]:
conf = 'ipsn'

In [4]:
ipsn_tpc = pd.read_csv('data/Clean_TPC/' + str(conf) + "_clean.csv")
ip_04 = ipsn_tpc[ipsn_tpc['Year'] == 2004]
# ip_04

In [5]:
uni_04 = collections.Counter(ip_04['University/Organization'])
df_04 = pd.DataFrame(uni_04.items(), columns=['University/Organization', 'Count'])
df_04 = df_04.sort_values(by = ['Count'], ascending = False)
len(df_04) * 0.20
# df_04


7.800000000000001

In [6]:
int(7.888)

7

In [10]:
t= df_04[:7]
t['Count'].sum()

25

In [ ]:
def 